# Initialize Lakehouse Names

In [ ]:
raw_lakehouse = "lh_bronze"
clean_lakehouse = "lh_gold"

# Set Git URLs

In [ ]:
GITHUB_LINK = "https://github.com/akhilannan/Fabric-Cricsheet"
RAW_PATH = "/raw/main"
SEMANTIC_MODEL = "Cricsheet Model"
REPORT_NAME = "Cricsheet Analysis"
BASE_PATH = GITHUB_LINK + RAW_PATH
BIM_URL = f"{BASE_PATH}/Semantic Model/{SEMANTIC_MODEL}.bim"
REPORT_URL = f"{BASE_PATH}/Power%20BI%20Report/{REPORT_NAME}.Report/report.json"
NOTEBOOK_URL = f"{BASE_PATH}/Notebooks"
UTILS_URL = f"{NOTEBOOK_URL}/Fabric%20Utilities.ipynb"

# Initialize Common Functions and Libraries

In [ ]:
import requests

def download_notebook_code(url):
    # Get the JSON representation of the notebook
    ipynb_json = requests.get(url).json()
    
    # Initialize variables to store the Python code and function names
    python_code = ""
    function_names = []

    # Iterate through the cells in the notebook
    for cell in ipynb_json['cells']:
        # Check if the cell is a code cell
        if cell['cell_type'] == 'code':
            # Extract the source code from the cell
            source_code = ''.join([
                f"get_ipython().system('{line.strip()[1:]}')" if line.strip().startswith('!pip install')
                else line for line in cell['source']
            ])
            
            # Add the source code to the Python code string
            python_code += source_code + '\n\n'
            
            # Find function definitions and add their names to the list
            function_names += [line.split()[1].split('(')[0] for line in cell['source'] if line.strip().startswith('def ')]

    # Return the Python code and the list of function names
    return python_code, function_names

python_code, function_names = download_notebook_code(UTILS_URL)
exec(python_code)

# Create Wheel Package

In [ ]:
import os
import subprocess
import sys
import shutil

def create_python_package(lakehouse, python_code, function_names):
    # Define the path to the directory where the Python file will be created
    package_path = os.path.join(get_lakehouse_path(lakehouse, "local", "Files"), "packages")
    fabric_path = os.path.join(package_path, "fabric_python_functions")
    fabric_utils_path = os.path.join(fabric_path, "fabric_utils")
    
    # Remove the existing package directory and create a new one
    shutil.rmtree(package_path, ignore_errors=True)
    os.makedirs(fabric_utils_path, exist_ok=True)

    # Write the Python code to the functions.py file
    python_file_path = os.path.join(fabric_utils_path, 'functions.py')
    with open(python_file_path, 'w') as python_file:
        python_file.write(python_code)

    # Write the __init__.py content
    init_content = "\n".join(f"from .functions import {name}" for name in function_names)
    init_file_path = os.path.join(fabric_utils_path, '__init__.py')
    with open(init_file_path, 'w') as init_file:
        init_file.write(init_content)

    # Write the setup.py content
    setup_content = """
from setuptools import setup, find_packages

setup(
    name='fabric_utils',
    version='0.1',
    packages=find_packages(),
    install_requires=['semantic-link'],
)
"""
    setup_file_path = os.path.join(fabric_path, 'setup.py')
    with open(setup_file_path, 'w') as setup_file:
        setup_file.write(setup_content)

    # Build the wheel file
    os.chdir(fabric_path)
    subprocess.check_call([sys.executable, 'setup.py', 'sdist', 'bdist_wheel'])

    # Move the wheel file outside the 'fabric_utils' folder
    wheel_files = [f for f in os.listdir('dist') if f.endswith('.whl')]
    for file in wheel_files:
        shutil.move(f'dist/{file}', f'../{file}')

    # Clean up the package directory
    shutil.rmtree(fabric_path)

    # Print a success message
    print("The wheel file has been successfully created and moved outside the 'fabric_utils' folder.")


# Create Lakehouse and Push wheel file to Files folder in Lakehouse

In [ ]:
for lakehouse in [raw_lakehouse, clean_lakehouse]:
    create_lakehouse_if_not_exists(lakehouse)
    create_python_package(lakehouse, python_code, function_names)

# Deploy Notebook

In [ ]:
# Use a function to handle the notebook processing
def process_notebook(file_name, default_lakehouse, notebook_url, code_replacements):
    file_name_w_extn = f"{file_name}.ipynb"
    link = f"{notebook_url}/{file_name_w_extn}"
    notebook_json = download_and_update_notebook(link, code_replacements)
    create_or_replace_notebook_from_ipynb(file_name, notebook_json, default_lakehouse)

# Define the code replacements using formatted strings
code_replacements = {
    'RAW_LAKEHOUSE = "lh_bronze"': f'RAW_LAKEHOUSE = "{raw_lakehouse}"',
    'CLEAN_LAKEHOUSE = "lh_gold"': f'CLEAN_LAKEHOUSE = "{clean_lakehouse}"'
}

# Define the mapping of notebook names to their respective default lakehouse
notebook_lakehouse = {
    'Cricsheet Orchestrator': raw_lakehouse,
    'Cricsheet Initialize': raw_lakehouse,
    'Cricsheet Ingest Data': raw_lakehouse,
    'Cricsheet Build Facts and Dimensions': clean_lakehouse,
    'Cricsheet Model Refresh': clean_lakehouse,
    'Cricsheet Optimize and Vacuum': raw_lakehouse
}

# Iterate over the notebook_lakehouse dictionary and process each notebook
for file_name, lakehouse_type in notebook_lakehouse.items():
    process_notebook(file_name, lakehouse_type, NOTEBOOK_URL, code_replacements)

# Deploy Semantic Model

In [ ]:
bim_json = requests.get(BIM_URL).json()
execute_with_retries(create_or_replace_semantic_model_from_bim,
                     dataset_name=SEMANTIC_MODEL,
                     bim_file_json=bim_json)

# Update Semantic Model Connection

In [ ]:
execute_with_retries(update_model_expression,
                     dataset_name=SEMANTIC_MODEL,
                     lakehouse_name=clean_lakehouse)

# Deploy Report and Repoint to the Semantic Model

In [ ]:
report_json = requests.get(REPORT_URL).json()
execute_with_retries(create_or_replace_report_from_reportjson,
                     report_name=REPORT_NAME,
                     dataset_name=SEMANTIC_MODEL,
                     report_json=report_json)